# URBANopt to Alfalfa workflow 

This script uploads OpenStudio models created from an URBANopt simulation to Alfalfa. It also allows setting actuator values as inputs to the corresponding EnergyPlus models and receiving sensor values as outputs. Such a workflow will enable interacting with the building energy models during run-time and implementing control strategies within an district or community. Refer to the [URBANopt to Alfalfa](https://github.com/NREL/alfalfa/wiki/Running-URBANopt-models-in-Alfalfa) documentation for more details.

In [1]:
import os
import shutil
import json
import pandas
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient
from pprint import pprint
from pathlib import Path

### Define alfalfa client object


In [2]:
ac = AlfalfaClient(host='https://alfalfa-staging.nrel.gov')

### Create folders to upload to Alfalfa from the URBANopt Project

The code below will create folders for each URBANopt building model that can be uploaded to Alfalfa. The folder contains: 

    - Model Folder: Contains OpenStudio model (.osm file) for each building created using the URBANopt simulation.
    - Measures Folder: Measures to be added while running Alfalfa in the OpenStudio Workflow
    - Weather Folder: Contains EPW weather file 
    - workflow.osw file: OpenStudio workflow file

**Define the following variables before running the code**:

- `uo_folder` : URBANopt Scenario Directory 
- `weather`: Define weather file used in URBANopt project
- `workflow`: OpenStudio workflow file name

In [10]:
# Set URBANopt scenario directory
uo_folder = Path('./example_urbanopt_project/run/example_urbanopt_scenario')

uo_scenario_folder = uo_folder.name
uo_scenario_path = Path(f"./{uo_scenario_folder}_alfalfa")
if uo_scenario_path.exists():
    shutil.rmtree(uo_scenario_path)
uo_scenario_path.mkdir(parents=True, exist_ok=True)

print(f"Folders saved at {uo_scenario_path}")

# Add .epw filename
weather = "USA_CO_Denver.Intl.AP.725650_TMY3.epw"

# Add .osw filename
workflow = "workflow.osw"

for file in uo_folder.iterdir():
        
    if file.is_dir() and (file / 'in.osm').exists():
        uo_model_filepath = uo_scenario_path / file.name
        uo_model_filepath.mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'models').mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'measures').mkdir(parents=True, exist_ok=True)
        (uo_model_filepath / 'weather').mkdir(parents=True, exist_ok=True)

        shutil.copy((file / 'in.osm'), (uo_model_filepath / 'models' / f'{file.name}.osm'))
        shutil.copy((uo_folder / '../../weather' / weather), (uo_model_filepath / 'weather'))
    
        osw = {"seed_file": f"{file.name}.osm",
           "weather_file": f"{weather}",
           "measure_paths": ["./measures"],
           "run_directory": "./run/",
           "file_paths": [
               "./weather/",
               "./models/"
           ],
            "steps" : [
                {
                    "measure_dir_name" : "alfalfa_vars",
                    "name" : "Alfalfa Variables",
                    "description" : "Add custom variables for Alfalfa",
                    "modeler_description" : "Add EMS global variables required by Alfalfa",
                    "arguments" : {
                        "model" : f"{file.name}.osm"
                    }
                }
            ]
          }
        
        f = open((uo_model_filepath / workflow), "w+")
        f.write(json.dumps(osw, indent=4))
        f.close()

print('Done')       

Folders saved at example_urbanopt_scenario_alfalfa
Done


In [41]:
model_path = './smalloffice'
print('done')

done


In [42]:
model_id = ac.submit(model_path) 
print(model_path)

Desired status: ready		Current status: preprocessing
Desired status: ready		Current status: ready
./smalloffice


### Upload models to Alfalfa

Submit models to Alfalfa server


In [11]:
files = list(uo_scenario_path.iterdir()) 
print(f"uploading {files}") 
model_ids = ac.submit(files) 
print(model_ids)


uploading [WindowsPath('example_urbanopt_scenario_alfalfa/61'), WindowsPath('example_urbanopt_scenario_alfalfa/62'), WindowsPath('example_urbanopt_scenario_alfalfa/63')]
Desired status: ready		Current status: preprocessing
Desired status: ready		Current status: preprocessing
Desired status: ready		Current status: preprocessing
Desired status: ready		Current status: ready
Desired status: ready		Current status: ready
Desired status: ready		Current status: ready
['758c4580-5345-11ee-80a8-f546b406f534', '7590b250-5345-11ee-80a8-f546b406f534', '758fc7f0-5345-11ee-80a8-f546b406f534']


### Define Alfalfa simulation parameters

- start_dt : This is the start date and time for the simulation in the following format (YYYY, M, D, H, M, S)
- end_dt : This is the end date and time for the simulation in the following format (YYYY, M, D, H, M, S)
- params : This defines the simulation parameters for the simulation including the start/end date and time and whether the model advances via API calls or a specified timescale
    

In [12]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2023, 5, 10, 12, 2, 0)
end_dt = datetime.datetime(2023, 5, 10, 12, 3, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": True,
    "start_datetime": start_dt,
    "end_datetime": end_dt,
    "timescale": 30
}

In [8]:
# Start Simulations
print(f"Starting site: {model_id}") 
ac.start(model_id, **params)
print('Done')

NameError: name 'model_id' is not defined

### Start simulations 

Start simulations as per the parameters defined

In [13]:
# Start Simulations
print(f"Starting site: {model_ids}") 
ac.start(model_ids, **params)
print('Done')

Starting site: ['758c4580-5345-11ee-80a8-f546b406f534', '7590b250-5345-11ee-80a8-f546b406f534', '758fc7f0-5345-11ee-80a8-f546b406f534']
Desired status: running		Current status: ready
Desired status: running		Current status: ready
Desired status: running		Current status: ready
Desired status: running		Current status: startedDesired status: running		Current status: started

Desired status: running		Current status: started
Desired status: running		Current status: running
Desired status: running		Current status: running
Desired status: running		Current status: running
Done


### Get the model's input points
Get a list of all of the model's input points (EP Actuators)
- model_id - the id of the site returned by the `ac.submit` function


In [16]:
# set model ID 
model_id = '758fc7f0-5345-11ee-80a8-f546b406f534'
print(f"{model_id} inputs:") 
pprint(ac.get_inputs(model_id))

758fc7f0-5345-11ee-80a8-f546b406f534 inputs:
['MasterEnable',
 'Zone_PVAV_with_PFP_Boxes_and_Reheat_1_Outside_Air_Damper_CMD',
 'Zone_PVAV_with_PFP_Boxes_and_Reheat_2_Outside_Air_Damper_CMD',
 'Zone_PVAV_with_PFP_Boxes_and_Reheat_3_Outside_Air_Damper_CMD',
 'Zone_PVAV_with_PFP_Boxes_and_Reheat_4_Outside_Air_Damper_CMD',
 'Zone_PVAV_with_PFP_Boxes_and_Reheat_Outside_Air_Damper_CMD',
 'OfficeMedium HTGSETP_SCH_YES_OPTIMUM',
 'OfficeMedium CLGSETP_SCH_YES_OPTIMUM']


### Set model input point

To set an input value use the `ac.set_inputs(model_id, inputs)` function.

- `inputs_dict` - a dictionary of input names and the desired values

In [17]:
input_dict = {'Zone_PVAV_with_PFP_Boxes_and_Reheat_1_Outside_Air_Damper_CMD': 10}
ac.set_inputs(model_id, input_dict)
print('Done')

Done


### Advance the model
12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.

In [ ]:
timesteps = 5
for _ in range(timesteps):
    ac.advance([model_id])
    print(f"Model advanced to time: {ac.get_sim_time(model_id)}")

print('Done')

### Get model's outputs
Query the outputs (EP Sensors) of the models as well as their values

In [18]:
print(f"{model_id} outputs:")
pprint(ac.get_outputs(model_id))

758fc7f0-5345-11ee-80a8-f546b406f534 outputs:
{'Building Story 1 ThermalZone 1 Air Temperature': 24,
 'Building Story 1 ThermalZone 1 Humidity': 20.4146964726179,
 'Building Story 1 ThermalZone 1 Temperature Setpoint': 24,
 'Building Story 1 ThermalZone 2 Air Temperature': 24,
 'Building Story 1 ThermalZone 2 Humidity': 21.95665528500485,
 'Building Story 1 ThermalZone 2 Temperature Setpoint': 24,
 'Building Story 1 ThermalZone 3 Air Temperature': 23.72126645674773,
 'Building Story 1 ThermalZone 3 Humidity': 21.90754018521439,
 'Building Story 1 ThermalZone 3 Temperature Setpoint': 23.71638567600263,
 'Building Story 1 ThermalZone 4 Air Temperature': 24,
 'Building Story 1 ThermalZone 4 Humidity': 19.95789269185017,
 'Building Story 1 ThermalZone 4 Temperature Setpoint': 24,
 'Building Story 1 ThermalZone 5 Air Temperature': 23.16152874619545,
 'Building Story 1 ThermalZone 5 Humidity': 22.78159382855611,
 'Building Story 1 ThermalZone 5 Temperature Setpoint': 23.1558779523121,
 'Buil

### Stop the simulations

In [ ]:
ac.stop(model_id)